In [10]:
from conllu import parse

with open('data/hungarian.conllu', 'r', encoding='utf8') as file:
    data = file.read()

sentences = parse(data)

import spacy
from spacy.tokens import Doc, Token
from conllu import parse_incr
from conllu import parse
from spacy.vocab import Vocab
from spacy import displacy
from pprint import pprint

nlp = spacy.blank("hu")  
vocab = nlp.vocab

if not Token.has_extension("xpos"):
    Token.set_extension("xpos", default=None)
if not Token.has_extension("deps"):
    Token.set_extension("deps", default=None)
if not Token.has_extension("misc"):
    Token.set_extension("misc", default=None)


def process_conllu_file(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for conllu_sentence in parse_incr(file):
            doc = conllu_to_spacy_doc(conllu_sentence, vocab)
            docs.append(doc)
    return docs

def conllu_to_spacy_doc(conllu_sentence, vocab):
    words = [token["form"] for token in conllu_sentence]
    heads = [token["head"] if token["head"] is not None else 0 for token in conllu_sentence]  # Head = 0 para raíces
    deps = [token["deprel"].split(":")[0] for token in conllu_sentence]
    lemmas = [token["lemma"] for token in conllu_sentence]
    pos_tags = [token["upostag"] for token in conllu_sentence]
    xpos_tags = [token["xpos"] for token in conllu_sentence]
    morph_feats = [token["feats"] or {} for token in conllu_sentence]
    misc_info = [token["misc"] or {} for token in conllu_sentence]
    deps_info = [token["deps"] or {} for token in conllu_sentence]

    # Crear el objeto Doc
    doc = Doc(vocab, words=words)

    # Asignar los atributos a cada token
    for i, token in enumerate(doc):
        token.lemma_ = lemmas[i]
        
        # Asignar el Universal POS a `pos_`, o usar `tag_` si no es válido
        if pos_tags[i] != "_":
            token.pos_ = pos_tags[i]
        elif xpos_tags[i] is not None:  # Verificar si xpos_tags[i] no es None antes de asignarlo
            token.tag_ = xpos_tags[i]  # Usar `xpos` como `tag_` si `upos` es "_"

        token.dep_ = deps[i]
        
        # Verificar que heads[i] sea un número entero antes de asignarlo como cabeza
        if isinstance(heads[i], int) and heads[i] > 0:
            token.head = doc[heads[i] - 1]  # Usar head como índice (spaCy usa índice 0 para la raíz)
        else:
            token.head = token  # Si head es 0 o no válido, el token se apunta a sí mismo como raíz

        # Asignar xpos, deps y misc como atributos personalizados
        token._.xpos = xpos_tags[i]
        token._.deps = deps_info[i]
        token._.misc = misc_info[i]

        # Convertir `feats` a un formato que spaCy entiende y asignarlo a morph
        if morph_feats[i]:
            feats_str = "|".join(f"{k}={v}" for k, v in morph_feats[i].items())
            token.set_morph(feats_str)

    return doc

def render_doc_by_sent_id(docs, sentences, target_sent_id):
    # Buscar el índice del `sent_id` deseado en los metadatos de `sentences`
    for i, sentence in enumerate(sentences):
        if sentence.metadata.get("sent_id") == target_sent_id:
            # Renderizar el Doc correspondiente usando `displacy`
            displacy.render(docs[i], style="dep", jupyter=True)
            return  # Salimos después de encontrar y renderizar
    print(f"Oración con sent_id '{target_sent_id}' no encontrada.")


file_path = "data/hungarian.conllu"
docs = process_conllu_file(file_path)



IN

In [5]:
render_doc_by_sent_id(docs, sentences, '1')
# The cat is in the wardrobe.

print(sentences[0])

for token in sentences[0]:
    print(token.items())

TokenList<A, macska, a, szekrényben, van, ., metadata={generator: "UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe", udpipe_model: "hungarian-szeged-ud-2.12-230717", udpipe_model_licence: "CC BY-NC-SA", newdoc: "None", newpar: "None", sent_id: "1", text: "A macska a szekrényben van."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '0:1'})])
dict_items([('id', 2), ('form', 'macska'), ('lemma', 'macska'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '2:8'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '9:10'})])
dict_items([('id', 4), ('form', 'szekré

In [7]:
render_doc_by_sent_id(docs, sentences, '2')
# She enters the shop.
# ground se glosa con nominativo

print(sentences[1])

for token in sentences[1]:
    print(token.items())

TokenList<Ő, bemegy, a, boltba, ., metadata={sent_id: "2", text: "Ő bemegy a boltba."}>
dict_items([('id', 1), ('form', 'Ő'), ('lemma', 'ő'), ('upos', 'PRON'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '28:29'})])
dict_items([('id', 2), ('form', 'bemegy'), ('lemma', 'bemegy'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '30:36'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '37:38'})])
dict_items([('id', 4), ('form', 'boltba'), ('lemma', 'bolt'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom',

In [8]:
render_doc_by_sent_id(docs, sentences, '3')
# He came out of the cave.


print(sentences[2])

for token in sentences[2]:
    print(token.items())

TokenList<Ő, kijött, a, barlangból, ., metadata={sent_id: "3", text: "Ő kijött a barlangból."}>
dict_items([('id', 1), ('form', 'Ő'), ('lemma', 'ő'), ('upos', 'PRON'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '47:48'})])
dict_items([('id', 2), ('form', 'kijött'), ('lemma', 'kijön'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '49:55'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '56:57'})])
dict_items([('id', 4), ('form', 'barlangból'), ('lemma', 'barlang'), ('upos', 'NOUN'), ('xpos', None), ('feats', {

ON

In [9]:
render_doc_by_sent_id(docs, sentences, '4')
# The book is on the table.

print(sentences[3])

for token in sentences[3]:
    print(token.items())

TokenList<A, könyv, az, asztalon, van, ., metadata={sent_id: "4", text: "A könyv az asztalon van."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '70:71'})])
dict_items([('id', 2), ('form', 'könyv'), ('lemma', 'könyv'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '72:77'})])
dict_items([('id', 3), ('form', 'az'), ('lemma', 'az'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '78:80'})])
dict_items([('id', 4), ('form', 'asztalon'), ('lemma', 'asztal'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Sup', 'Number': 'Sing'}), ('head', 5), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '81:89'

In [11]:
render_doc_by_sent_id(docs, sentences, '5')
# The boy jumped onto the table.


print(sentences[4])

for token in sentences[4]:
    print(token.items())

TokenList<A, fiú, felugrott, az, asztalra, ., metadata={sent_id: "5", text: "A fiú felugrott az asztalra."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '95:96'})])
dict_items([('id', 2), ('form', 'fiú'), ('lemma', 'fiú'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 3), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '97:100'})])
dict_items([('id', 3), ('form', 'felugrott'), ('lemma', 'fel+ugrik'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '101:110'})])
dict_items([('id', 4), ('form', 'az'), ('lemma', 'az'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'P

In [12]:
render_doc_by_sent_id(docs, sentences, '6')
# The girl climbed down from the table.


print(sentences[5])

for token in sentences[5]:
    print(token.items())

TokenList<A, lány, lemászott, az, asztalról, ., metadata={sent_id: "6", text: "A lány lemászott az asztalról."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '124:125'})])
dict_items([('id', 2), ('form', 'lány'), ('lemma', 'lány'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 3), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '126:130'})])
dict_items([('id', 3), ('form', 'lemászott'), ('lemma', 'lemászik'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '131:140'})])
dict_items([('id', 4), ('form', 'az'), ('lemma', 'az'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': '

AT

In [13]:
render_doc_by_sent_id(docs, sentences, '7')
# The bird rests next to the tree.


print(sentences[6])

for token in sentences[6]:
    print(token.items())


TokenList<A, madár, a, fánál, pihen, ., metadata={sent_id: "7", text: "A madár a fánál pihen."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '155:156'})])
dict_items([('id', 2), ('form', 'madár'), ('lemma', 'madár'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '157:162'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '163:164'})])
dict_items([('id', 4), ('form', 'fánál'), ('lemma', 'fánál'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '165:170'

In [14]:
render_doc_by_sent_id(docs, sentences, '8')
# The boy runs toward the tree.

print(sentences[7])

for token in sentences[7]:
    print(token.items())

TokenList<A, fiú, a, fához, fut, ., metadata={sent_id: "8", text: "A fiú a fához fut."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '178:179'})])
dict_items([('id', 2), ('form', 'fiú'), ('lemma', 'fiú'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '180:183'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '184:185'})])
dict_items([('id', 4), ('form', 'fához'), ('lemma', 'fz'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'All', 'Number': 'Sing'}), ('head', 5), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '186:191'})])
dict_items([

In [15]:
render_doc_by_sent_id(docs, sentences, '9')
# The squirrel moved away from the tree.


print(sentences[8])

for token in sentences[8]:
    print(token.items())

TokenList<A, mókus, eltávolodott, a, fától, ., metadata={sent_id: "9", text: "A mókus eltávolodott a fától."}>
dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '197:198'})])
dict_items([('id', 2), ('form', 'mókus'), ('lemma', 'mókus'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 3), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '199:204'})])
dict_items([('id', 3), ('form', 'eltávolodott'), ('lemma', 'el+távolodik'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '205:217'})])
dict_items([('id', 4), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definit